In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.
print(data_dev.shape)

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape
print(data_train.shape)

(785, 1000)
(785, 41000)


In [79]:
def init_params():
    W1 = np.random.rand(200,784) - 0.5
    b1 = np.random.rand(200,1) - 0.5
    W2 = np.random.rand(200,200) - 0.5
    b2 = np.random.rand(200,1) - 0.5
    W3 = np.random.rand(10,200) - 0.5
    b3 = np.random.rand(10,1) - 0.5
    return W1,b1,W2,b2,W3,b3

def ReLU(Z):
    return np.maximum(0,Z)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(X,W1,b1,W2,b2,W3,b3):
    Z1 = W1.dot(X)+b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1)+b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2)+b3
    A3 = softmax(Z3)
    return Z1,A1,Z2,A2,Z3,A3

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z>0

def back_prop(Y,X,W1,b1,W2,b2,W3,b3,A1,Z1,A2,Z2,A3,Z3):
    dZ3 = A3 - one_hot(Y)
    dW3 = 1/m * dZ3.dot(A2.T)
    db3 = 1/m * np.sum(dZ3)
    dZ2 = W3.T.dot(dZ3) * deriv_ReLU(Z2)
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1)
    return dZ3,dW3,db3,dZ2,dW2,db2,dZ1,dW1,db1

def update_params(W3,b3,W2,b2,W1,b1,dW3,db3,dW2,db2,dW1,db1,alpha):
    W3 = W3 - alpha*dW3
    b3 = b3 - alpha*db3
    W2 = W2 - alpha*dW2
    b2 = b2 - alpha*db2
    W1 = W1 - alpha*dW1
    b1 = b1 - alpha*db1
    return W3,b3,W2,b2,W1,b1
    
    

In [80]:
def get_predictions(A3):
    return np.argmax(A3, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_desc(X,Y,iters,alpha):
    W1,b1,W2,b2,W3,b3 = init_params()
    
    for i in range(iters):
        Z1,A1,Z2,A2,Z3,A3 = forward_prop(X,W1,b1,W2,b2,W3,b3)
        dZ3,dW3,db3,dZ2,dW2,db2,dZ1,dW1,db1 = back_prop(Y,X,W1,b1,W2,b2,W3,b3,A1,Z1,A2,Z2,A3,Z3)
        W3,b3,W2,b2,W1,b1 = update_params(W3,b3,W2,b2,W1,b1,dW3,db3,dW2,db2,dW1,db1,alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A3)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2, W3, b3


In [81]:
W1,b1,W2,b2,W3,b3 = gradient_desc(X_train,Y_train,600,0.1)

Iteration:  0
[5 7 2 ... 5 9 7] [9 7 3 ... 8 6 7]
0.10412195121951219
Iteration:  10
[7 7 9 ... 9 4 9] [9 7 3 ... 8 6 7]
0.2959512195121951
Iteration:  20
[7 7 9 ... 2 6 7] [9 7 3 ... 8 6 7]
0.3755121951219512
Iteration:  30
[7 7 9 ... 2 6 7] [9 7 3 ... 8 6 7]
0.44495121951219513
Iteration:  40
[7 7 9 ... 2 6 7] [9 7 3 ... 8 6 7]
0.5111219512195122
Iteration:  50
[7 7 9 ... 8 6 7] [9 7 3 ... 8 6 7]
0.5734146341463414
Iteration:  60
[7 7 9 ... 8 6 7] [9 7 3 ... 8 6 7]
0.6249024390243902
Iteration:  70
[9 7 9 ... 8 6 7] [9 7 3 ... 8 6 7]
0.6664390243902439
Iteration:  80
[9 7 9 ... 8 6 7] [9 7 3 ... 8 6 7]
0.6996585365853658
Iteration:  90
[9 7 9 ... 8 6 7] [9 7 3 ... 8 6 7]
0.7233414634146341
Iteration:  100
[9 7 9 ... 8 6 7] [9 7 3 ... 8 6 7]
0.7431951219512195
Iteration:  110
[9 7 9 ... 8 6 7] [9 7 3 ... 8 6 7]
0.7582926829268293
Iteration:  120
[9 7 9 ... 8 6 7] [9 7 3 ... 8 6 7]
0.7710243902439025
Iteration:  130
[9 7 9 ... 8 6 7] [9 7 3 ... 8 6 7]
0.7825121951219512
Iteration:  140

In [82]:
from matplotlib import pyplot as plt

def make_predictions(X, W1, b1, W2, b2, W3, b3):
    _, _, _, _, _, A3 = forward_prop(X, W1, b1, W2, b2, W3, b3)
    predictions = get_predictions(A3)
    return predictions

def test_prediction(index, W1, b1, W2, b2, W3, b3):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2, W3, b3)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [83]:
dev_predictions = make_predictions(X_dev,W1, b1, W2, b2, W3, b3)
get_accuracy(dev_predictions,Y_dev)

[2 9 8 7 3 8 5 7 6 7 4 5 1 5 5 0 2 7 1 9 3 2 6 6 9 7 6 2 7 6 3 9 1 8 0 9 3
 1 3 4 7 5 6 1 9 6 5 7 4 1 7 9 8 5 7 7 9 6 8 8 6 1 2 1 6 0 6 8 5 4 4 1 9 0
 1 7 2 3 0 3 7 7 1 8 9 8 4 8 5 0 4 4 8 9 7 3 0 9 6 5 5 1 5 6 7 1 9 4 9 0 9
 3 8 4 9 0 8 5 2 1 3 3 9 5 4 3 6 3 3 8 1 5 4 2 8 5 1 0 3 6 8 0 7 3 7 1 7 3
 5 6 7 1 6 3 7 4 6 4 7 3 8 9 4 9 1 1 2 7 1 2 4 3 5 0 2 2 4 0 6 3 5 6 1 6 7
 4 8 5 4 5 7 6 1 5 8 3 6 6 5 6 5 7 2 8 0 6 4 7 1 0 9 4 4 9 3 7 6 1 9 1 2 9
 6 2 7 2 8 9 4 2 0 4 3 0 5 4 2 9 3 5 1 9 7 1 9 9 6 2 8 7 7 1 8 5 0 9 5 2 3
 4 4 8 1 0 0 6 3 5 3 4 3 0 8 0 9 3 5 4 9 8 6 8 8 6 2 5 0 0 1 5 8 1 9 3 9 2
 1 9 7 1 5 6 5 0 8 9 1 3 1 1 9 7 1 0 0 5 3 4 0 1 8 5 4 8 4 3 0 6 3 0 0 7 2
 4 2 2 7 8 5 0 3 7 3 1 1 5 3 7 4 4 4 3 5 5 8 0 3 9 8 1 4 4 6 8 5 2 5 1 4 7
 0 9 0 7 9 2 0 0 8 6 7 1 4 8 2 4 7 4 5 2 0 3 7 8 4 1 8 8 0 7 2 4 6 4 4 9 3
 7 1 6 9 5 0 1 0 6 9 0 3 2 4 3 5 3 8 7 5 9 0 0 1 9 6 5 2 5 6 4 2 4 1 6 0 3
 2 8 1 7 6 4 3 2 5 3 8 5 3 3 2 8 5 8 0 8 7 5 4 0 2 0 8 4 3 1 8 4 9 7 5 2 6
 5 9 9 1 4 3 7 8 2 1 2 1 

0.9